# CBS

In [13]:
# all imports
import pandas as pd
import os
import instaloader
from instaloader import Post
import sys
import re

In [61]:
# functions    
def importCSV(filename):
    return pd.read_csv(filename, delimiter=';')

def outputCSV(dataset, filename):
    dataset.to_csv("./../data/" + filename, sep=';')
    
def cprint(text):
    sys.stdout.write("\r" + text)
    sys.stdout.flush()

In [60]:
# retrieve data
insta = importCSV('D:/Data/School/S7/Proftaak/Code Base/CBS/data/Coosto_berichten.csv')

# remove empty columns
insta = insta.drop(['zoekopdracht', 'type', 'volgers', 'invloed', 'titel', 'type bron'], axis=1)

# check with count()
insta.count()

AttributeError: 'NoneType' object has no attribute 'drop'

In [57]:
# check if post still exists
indexes_to_drop = []
total_length = len(insta)


def check_post(url, index):
    try:
        L = instaloader.Instaloader()
        post = Post.from_shortcode(L.context, url.split("/")[-2])
        insta.at[index, 'likes count'] = post.likes
        
    except:
        # When exception is thrown add index to list of indexes that need to be droppped
        indexes_to_drop.append(index)

def del_posts(data, indexes_to_drop):
    for index in indexes_to_drop:
        data = data.drop(index=index, axis=0)
    return data

for index, row in insta.iterrows():
    check_post(row['url'], index)
    cprint(str(round((index/total_length)*100)) + "%")
cprint(str(100) + "%")
print('\nUnvalid urls found', len(indexes_to_drop))
    
# delete all non existing posts from dataset
insta = del_posts(insta, indexes_to_drop)

insta.index = range(len(insta))
insta.head(22)

0%

5%

9%

14%

18%

...

91%

95%

100%


Unvalid urls found 3


,datum,url,sentiment,discussielengte,views,auteur,GPS breedtegraad,GPS lengtegraad,bericht tekst,likes count
0,2017-06-20 17:06,https://instagram.com/p/BVjjp3Cl4uK/,NaN,9.0,NaN,marusinalavka,NaN,NaN,"@_korobo_ кстати хочу сказать, что коробочки ...",31.0
1,2017-10-10 21:43,https://instagram.com/p/BaFFd0fBbcw/,NaN,64.0,NaN,artofobservance,NaN,NaN,So chic,3639.0
2,2017-04-28 16:47,https://instagram.com/p/BTbF7AEDgHg/,NaN,617.0,NaN,amigabali,NaN,NaN,Incredible,534004.0
3,2017-08-30 17:43,https://instagram.com/p/BYa-vCWnF-U/,+,20.0,NaN,healthy.sneakers,NaN,NaN,😄😄👍,305.0
4,2017-09-04 22:22,https://instagram.com/p/BYoVU-GFypi/,NaN,20.0,NaN,gpazminoyepez,NaN,NaN,@divaribas.82 aloe y fibern plus. Escribame 09...,2224.0
5,2017-11-08 10:29,https://instagram.com/p/BbOnSBlFwmo/,+,10.0,NaN,laurienblomphotography,NaN,NaN,"@remkevet oooh wat goed haha, heerlijk!:)",46.0
6,2017-07-20 08:02,https://instagram.com/p/BWwfFxkFoHW/,NaN,NaN,10.0,breaking_glass,NaN,NaN,Lovely Japanese copy of Pin Ups. One of my fav...,63.0
7,2018-02-10 22:27,https://instagram.com/p/Bdp4B0Pj6TL/,NaN,3124.0,NaN,belmacurkicc,NaN,NaN,@amanda.carlberg,223473.0
8,2018-02-21 18:01,https://instagram.com/p/BfdzxFYlj-h/,+,596.0,NaN,_lilysmith97_,NaN,NaN,@nobbsy91 look at this!😋,103847.0
9,2017-08-28 17:40,https://instagram.com/p/BYVy0NRBGdr/,NaN,2.0,NaN,heimint,NaN,NaN,난 맨날 아쉬워서 밤을 새ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ아 🐶피곤.......다크써클#지못미,17.0


In [54]:
def remove_emoji(data):  
    indexes_to_drop = []
    for index, row in data.iterrows():
        a = row['bericht tekst']

        # todo: vul aan met meer emoji's
        emoji_pattern = re.compile("["
                          u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
        newValue = emoji_pattern.sub(r'', a)
        if newValue == '' : 
            indexes_to_drop.append(newValue)
        else:
            data.at[index, 'bericht tekst'] = newValue
            
    return data

insta = remove_emoji(insta)
insta

,datum,url,sentiment,discussielengte,views,auteur,GPS breedtegraad,GPS lengtegraad,bericht tekst,likes count,hashtags
0,2017-06-20 17:06,https://instagram.com/p/BVjjp3Cl4uK/,NaN,9.0,NaN,marusinalavka,NaN,NaN,"@_korobo_ кстати хочу сказать, что коробочки ...",31.0,
1,2017-10-10 21:43,https://instagram.com/p/BaFFd0fBbcw/,NaN,64.0,NaN,artofobservance,NaN,NaN,So chic,3639.0,
2,2017-04-28 16:47,https://instagram.com/p/BTbF7AEDgHg/,NaN,617.0,NaN,amigabali,NaN,NaN,Incredible,534004.0,
3,2017-08-30 17:43,https://instagram.com/p/BYa-vCWnF-U/,+,20.0,NaN,healthy.sneakers,NaN,NaN,😄😄👍,305.0,
4,2017-09-04 22:22,https://instagram.com/p/BYoVU-GFypi/,NaN,20.0,NaN,gpazminoyepez,NaN,NaN,@divaribas.82 aloe y fibern plus. Escribame 09...,2224.0,
5,2017-11-08 10:29,https://instagram.com/p/BbOnSBlFwmo/,+,10.0,NaN,laurienblomphotography,NaN,NaN,"@remkevet oooh wat goed haha, heerlijk!:)",46.0,
6,2017-07-20 08:02,https://instagram.com/p/BWwfFxkFoHW/,NaN,NaN,10.0,breaking_glass,NaN,NaN,Lovely Japanese copy of Pin Ups. One of my fav...,63.0,
7,2018-02-10 22:27,https://instagram.com/p/Bdp4B0Pj6TL/,NaN,3124.0,NaN,belmacurkicc,NaN,NaN,@amanda.carlberg,223473.0,
8,2018-02-21 18:01,https://instagram.com/p/BfdzxFYlj-h/,+,596.0,NaN,_lilysmith97_,NaN,NaN,@nobbsy91 look at this!,103847.0,
9,2017-08-28 17:40,https://instagram.com/p/BYVy0NRBGdr/,NaN,2.0,NaN,heimint,NaN,NaN,.......#,17.0,


In [55]:
def isolate_hashtag(data):  
    for index, row in data.iterrows():
        text = row['bericht tekst']
        
        # find all hashtags in text and isolate them in new column
        hashtags = re.findall(r"#(\w+)", text)
        print(text)
        print(hashtags)
        if(len(hashtags) > 0):
            data.at[index, 'hashtags'] = hashtags
        
        #remove hashtags from text
        pattern = re.compile("#(\w+)")
        newText = pattern.sub(r'', text)
        data.at[index, 'bericht tekst'] = newText
        
    return data

result = isolate_hashtag(insta)
result

@_korobo_  кстати хочу сказать, что коробочки снимать гораздо сложней.  Я пробовала свои отснять. Не просто, так сказать....
[]
So chic
[]
Incredible
[]
😄😄👍
[]
@divaribas.82 aloe y fibern plus. Escribame 0994895191
[]
@remkevet oooh wat goed haha, heerlijk!:)
[]
Lovely Japanese copy of Pin Ups. One of my favourites - soon after dark Emily cry's.     
[]
@amanda.carlberg
[]
@nobbsy91  look at this!
[]
     .......#
[]
I can't wait to spend this weekend at  - -  to Wondercon last year when I ran into @brucewayne626's amazing Armored Batman.
[]
Schönen Abend für Euch 
[]
Het programma word steeds slechter!!! Het nieuwe format is al niet om aan te zien, nu ook nog zo dramatische patatbakker als coach? Zonde van het programma  
[]
Fancy salad 🥗                              
[]
@teamdavidneres dus dan is hij niet meteen Een prutser hij is beter dan jou
[]
En manga larga y estan a 40 grados en madrid @irenedosil
[]
@jakenorth03
[]
I love you I reel completely grateful to you
[]
Wow!
[]


,datum,url,sentiment,discussielengte,views,auteur,GPS breedtegraad,GPS lengtegraad,bericht tekst,likes count,hashtags
0,2017-06-20 17:06,https://instagram.com/p/BVjjp3Cl4uK/,NaN,9.0,NaN,marusinalavka,NaN,NaN,"@_korobo_ кстати хочу сказать, что коробочки ...",31.0,
1,2017-10-10 21:43,https://instagram.com/p/BaFFd0fBbcw/,NaN,64.0,NaN,artofobservance,NaN,NaN,So chic,3639.0,
2,2017-04-28 16:47,https://instagram.com/p/BTbF7AEDgHg/,NaN,617.0,NaN,amigabali,NaN,NaN,Incredible,534004.0,
3,2017-08-30 17:43,https://instagram.com/p/BYa-vCWnF-U/,+,20.0,NaN,healthy.sneakers,NaN,NaN,😄😄👍,305.0,
4,2017-09-04 22:22,https://instagram.com/p/BYoVU-GFypi/,NaN,20.0,NaN,gpazminoyepez,NaN,NaN,@divaribas.82 aloe y fibern plus. Escribame 09...,2224.0,
5,2017-11-08 10:29,https://instagram.com/p/BbOnSBlFwmo/,+,10.0,NaN,laurienblomphotography,NaN,NaN,"@remkevet oooh wat goed haha, heerlijk!:)",46.0,
6,2017-07-20 08:02,https://instagram.com/p/BWwfFxkFoHW/,NaN,NaN,10.0,breaking_glass,NaN,NaN,Lovely Japanese copy of Pin Ups. One of my fav...,63.0,
7,2018-02-10 22:27,https://instagram.com/p/Bdp4B0Pj6TL/,NaN,3124.0,NaN,belmacurkicc,NaN,NaN,@amanda.carlberg,223473.0,
8,2018-02-21 18:01,https://instagram.com/p/BfdzxFYlj-h/,+,596.0,NaN,_lilysmith97_,NaN,NaN,@nobbsy91 look at this!,103847.0,
9,2017-08-28 17:40,https://instagram.com/p/BYVy0NRBGdr/,NaN,2.0,NaN,heimint,NaN,NaN,.......#,17.0,


In [41]:
# output new cleaned dataset
data = insta
outputCSV(data, "cleaned.csv")